In [1]:
import tensor_comprehensions as tc
import torch
import tc_composer
from torch.nn import Conv2d
from tc_composer.implementation import AlexNetConv0, AlexNetConv1, AlexNetConv2, AlexNetConv3, AlexNetConv4, AlexNetConv5, AlexNetConv6, AlexNetConv7

[INFO] tc_composer.settings - Setting default tensor type: torch.cuda.FloatTensor
[INFO] tc_composer.settings - Setting epsilon: 1e-16
[INFO] tc_composer.settings - Input tensor shape checking: False
[INFO] tc_composer.settings - Saving compiled options in: /home/ubuntu/tc_composer/options


# AlexNet
Implementing https://github.com/pytorch/vision/blob/master/torchvision/models/alexnet.py

In [2]:
num_classes = 1000

In [3]:
batch_size = 1
in_channels, in_height, in_width = 3, 227, 227
tc_inp = tc_image = torch.randn(batch_size, in_channels, in_height, in_width)

In [4]:
Layers = (AlexNetConv0, AlexNetConv1, AlexNetConv2, AlexNetConv3, AlexNetConv4, AlexNetConv5, AlexNetConv6, AlexNetConv7)
layers = tuple(l() for l in Layers)

In [5]:
def compile_():
    inp = tc_inp
    for l in layers:
        l.recompile(inp)
        inp = l(inp)

compile_()

[INFO] AlexNetConv0 - Compiling for input shape - [(1, 3, 227, 227)].
[WARNING] AlexNetConv0 - No option loaded from file for input shape - [(1, 3, 227, 227)].
[WARNING] AlexNetConv0 - Initializing naive options.
[INFO] AlexNetConv1 - Compiling for input shape - [(1, 64, 56, 56)].
[WARNING] AlexNetConv1 - No option loaded from file for input shape - [(1, 64, 56, 56)].
[WARNING] AlexNetConv1 - Initializing naive options.
[INFO] AlexNetConv2 - Compiling for input shape - [(1, 64, 27, 27)].
[WARNING] AlexNetConv2 - No option loaded from file for input shape - [(1, 64, 27, 27)].
[WARNING] AlexNetConv2 - Initializing naive options.
[INFO] AlexNetConv3 - Compiling for input shape - [(1, 192, 27, 27)].
[WARNING] AlexNetConv3 - No option loaded from file for input shape - [(1, 192, 27, 27)].
[WARNING] AlexNetConv3 - Initializing naive options.
[INFO] AlexNetConv4 - Compiling for input shape - [(1, 192, 13, 13)].
[WARNING] AlexNetConv4 - No option loaded from file for input shape - [(1, 192, 13

# Correctness

In [6]:
from torchvision.models import AlexNet

torch_alex_net = AlexNet()
torch_alex_net.train(False)  # Disable dropout

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

In [7]:
from itertools import chain

for p, p1 in zip(torch_alex_net.features.parameters(), chain(*(l.params for l in layers))):
    p.data = p1.data

In [8]:
def run(inp):
    out = inp
    for l in layers:
        out = l(out)
    return out

In [9]:
import numpy as np

np.testing.assert_allclose(run(tc_inp).data, torch_alex_net.features(tc_inp).data, rtol=1e-4)